In [1]:
'''
Herein, association mining based on online reatil sale is performed
'''
import pandas as pd

In [2]:
#load the dataset
#Due to the error "utf-8 codec can't decode byte 0xa3 in position 28: invalid start byte"
#encoding = 'unicode_escape' is applied
df = pd.read_csv('casestudy2-data/online retail.csv',header=0,encoding = 'unicode_escape')

C:\Users\kmw41\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532619 entries, 0 to 532618
Data columns (total 8 columns):
InvoiceNo      532619 non-null object
StockCode      532619 non-null object
Description    531165 non-null object
Quantity       532619 non-null int64
InvoiceDate    532619 non-null object
UnitPrice      532619 non-null float64
CustomerID     397924 non-null float64
Country        532619 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 32.5+ MB


In [4]:
'''
Data preprocessing 

### do in need to remove space?###
'''
#remove negative values in quantiy column as it is not necessaory values in terms of analysis of association.
df.drop(df[df.Quantity < 0].index, inplace=True)
    
# remove inappropriate values
df.drop(df[df.StockCode == 'DCGS0073'].index, inplace=True) # written as ebay
df.drop(df[df.StockCode == 'DCGS0069'].index, inplace=True) # written as ebay
df.drop(df[df.StockCode == 'DCGS0068'].index, inplace=True) # written as ebay
df.drop(df[df.StockCode == 'DCGS0067'].index, inplace=True) # written as ebay
df.drop(df[df.StockCode == 'DCGS0003'].index, inplace=True) # written as ebay
df.drop(df[df.StockCode == 'B'].index, inplace=True)
df.drop(df[df.StockCode == 'AMAZONEFEE'].index, inplace=True)
df.drop(df[df.StockCode == 'BANK CHARGES'].index, inplace=True)
df.drop(df[df.StockCode == 'C2'].index, inplace=True)
df.drop(df[df.StockCode == 'DOT'].index, inplace=True)
df.drop(df[df.StockCode == 'M'].index, inplace=True)
df.drop(df[df.StockCode == 'PADS'].index, inplace=True)
df.drop(df[df.StockCode == 'POST'].index, inplace=True)
df.drop(df[df.StockCode == 'S'].index, inplace=True)

#gift is a gift voucher so that it may be better not to remove it
#df.drop(df[df.StockCode == 'gift_0001_10'].index, inplace=True)
#df.drop(df[df.StockCode == 'gift_0001_20'].index, inplace=True)
#df.drop(df[df.StockCode == 'gift_0001_30'].index, inplace=True)
#df.drop(df[df.StockCode == 'gift_0001_40'].index, inplace=True)
#df.drop(df[df.StockCode == 'gift_0001_50'].index, inplace=True)
 


In [5]:
#df = df.groupby(['InvoiceNo'])['StockCode'].apply(list)
df = df.dropna(subset = ['Description'])
#MissingData = df.isnull().sum()
#print(MissingData.sort_values(ascending = False))

In [6]:
#print(df['Description'].unique())
#print(df['Description'].value_counts())

In [7]:
transactions = df.groupby(['InvoiceNo'])['Description'].apply(list)
print(transactions.head(5))

InvoiceNo
536365    [WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...
536366    [HAND WARMER UNION JACK, HAND WARMER RED POLKA...
536367    [ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO...
536368    [JAM MAKING SET WITH JARS, RED COAT RACK PARIS...
536369                           [BATH BUILDING BLOCK WORD]
Name: Description, dtype: object


In [8]:
print(transactions.shape)

(19958,)


In [9]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift']) 

In [58]:
from apyori import apriori

# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.009, min_confidence = 0.2, min_lift = 3, min_length = 2, max_length = 2))

# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'JAM MAKING SET PRINTED', '6 RIBBONS RUSTIC CHARM'}), support=0.01182483214750977, ordered_statistics=[OrderedStatistic(items_base=frozenset({'6 RIBBONS RUSTIC CHARM'}), items_add=frozenset({'JAM MAKING SET PRINTED'}), confidence=0.24712041884816752, lift=4.2444314280307465), OrderedStatistic(items_base=frozenset({'JAM MAKING SET PRINTED'}), items_add=frozenset({'6 RIBBONS RUSTIC CHARM'}), confidence=0.20309810671256454, lift=4.2444314280307465)]), RelationRecord(items=frozenset({'JAM MAKING SET WITH JARS', '6 RIBBONS RUSTIC CHARM'}), support=0.010371780739553062, ordered_statistics=[OrderedStatistic(items_base=frozenset({'6 RIBBONS RUSTIC CHARM'}), items_add=frozenset({'JAM MAKING SET WITH JARS'}), confidence=0.21675392670157068, lift=3.8181596373432902)]), RelationRecord(items=frozenset({'NATURAL SLATE HEART CHALKBOARD ', '6 RIBBONS RUSTIC CHARM'}), support=0.009920833750876842, ordered_statistics=[OrderedStatistic(items_base=frozenset({'6 RIBBONS RUS

In [59]:
result_df = convert_apriori_results_to_pandas_df(results)
print(result_df.head(20))

                             Left_side                          Right_side  \
0               6 RIBBONS RUSTIC CHARM              JAM MAKING SET PRINTED   
1               JAM MAKING SET PRINTED              6 RIBBONS RUSTIC CHARM   
2               6 RIBBONS RUSTIC CHARM            JAM MAKING SET WITH JARS   
3               6 RIBBONS RUSTIC CHARM     NATURAL SLATE HEART CHALKBOARD    
4               6 RIBBONS RUSTIC CHARM     PACK OF 72 RETROSPOT CAKE CASES   
5               6 RIBBONS RUSTIC CHARM     RECIPE BOX PANTRY YELLOW DESIGN   
6               6 RIBBONS RUSTIC CHARM           SCANDINAVIAN REDS RIBBONS   
7            SCANDINAVIAN REDS RIBBONS              6 RIBBONS RUSTIC CHARM   
8               6 RIBBONS RUSTIC CHARM   SET OF 3 CAKE TINS PANTRY DESIGN    
9      60 CAKE CASES DOLLY GIRL DESIGN         60 TEATIME FAIRY CAKE CASES   
10         60 TEATIME FAIRY CAKE CASES     60 CAKE CASES DOLLY GIRL DESIGN   
11     60 CAKE CASES DOLLY GIRL DESIGN  PACK OF 60 PINK PAISLEY 

In [60]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(15))

                Left_side            Right_side   Support  Confidence  \
424     HERB MARKER THYME  HERB MARKER ROSEMARY  0.011073    0.932489   
423  HERB MARKER ROSEMARY     HERB MARKER THYME  0.011073    0.920833   
407   HERB MARKER CHIVES    HERB MARKER PARSLEY  0.009670    0.923445   
408   HERB MARKER PARSLEY   HERB MARKER CHIVES   0.009670    0.810924   
412     HERB MARKER THYME   HERB MARKER CHIVES   0.009470    0.797468   
411   HERB MARKER CHIVES      HERB MARKER THYME  0.009470    0.904306   
405   HERB MARKER CHIVES       HERB MARKER MINT  0.009520    0.909091   
406      HERB MARKER MINT   HERB MARKER CHIVES   0.009520    0.791667   
422     HERB MARKER THYME   HERB MARKER PARSLEY  0.010672    0.898734   
421   HERB MARKER PARSLEY     HERB MARKER THYME  0.010672    0.894958   
409   HERB MARKER CHIVES   HERB MARKER ROSEMARY  0.009470    0.904306   
410  HERB MARKER ROSEMARY   HERB MARKER CHIVES   0.009470    0.787500   
419   HERB MARKER PARSLEY  HERB MARKER ROSEMARY  0.

In [63]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Confidence', ascending=False)
print(result_df.head(15))

                   Left_side                Right_side   Support  Confidence  \
424        HERB MARKER THYME      HERB MARKER ROSEMARY  0.011073    0.932489   
407      HERB MARKER CHIVES        HERB MARKER PARSLEY  0.009670    0.923445   
423     HERB MARKER ROSEMARY         HERB MARKER THYME  0.011073    0.920833   
1315  REGENCY TEA PLATE PINK  REGENCY TEA PLATE GREEN   0.013929    0.911475   
405      HERB MARKER CHIVES           HERB MARKER MINT  0.009520    0.909091   
411      HERB MARKER CHIVES          HERB MARKER THYME  0.009470    0.904306   
409      HERB MARKER CHIVES       HERB MARKER ROSEMARY  0.009470    0.904306   
419      HERB MARKER PARSLEY      HERB MARKER ROSEMARY  0.010773    0.903361   
422        HERB MARKER THYME       HERB MARKER PARSLEY  0.010672    0.898734   
420     HERB MARKER ROSEMARY       HERB MARKER PARSLEY  0.010773    0.895833   
421      HERB MARKER PARSLEY         HERB MARKER THYME  0.010672    0.894958   
402     HERB MARKER ROSEMARY         HER

In [61]:
search = result_df[result_df['Left_side'] == 'HERB MARKER CHIVES ']
print(search)

               Left_side            Right_side   Support  Confidence  \
407  HERB MARKER CHIVES    HERB MARKER PARSLEY  0.009670    0.923445   
411  HERB MARKER CHIVES      HERB MARKER THYME  0.009470    0.904306   
405  HERB MARKER CHIVES       HERB MARKER MINT  0.009520    0.909091   
409  HERB MARKER CHIVES   HERB MARKER ROSEMARY  0.009470    0.904306   
396  HERB MARKER CHIVES      HERB MARKER BASIL  0.009119    0.870813   

          Lift  
407  77.437457  
411  76.152504  
405  75.598485  
409  75.200598  
396  71.816916  
